In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import packages and formatting statements

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join, Column
from astropy.wcs import WCS
from astropy.io import ascii
import os

plt.rc('text',usetex=False)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

In [3]:
# Compile galaxy and OSC objects
# full catalog header: Name,Type,Host,RA hms,Dec dms,RA dds,Dec dds,DiscDate

DataFile = '../Data/0.MUSEdata.csv'
Data = Table.read(DataFile, format='csv') 
galaxies, dists =  Data["Galaxy"], Data["Distance"]
COPTMaps,maps150, maps1kpc = Data["SFRcopt"],Data["maps150"],Data["maps1kpc"]
SNe, types = np.loadtxt('../Data/1.FullCatalog.csv', dtype = str, delimiter = ",",unpack=True, usecols = (0,1))
ras, decs = np.loadtxt('../Data/1.FullCatalog.csv',delimiter=',', unpack = True, usecols=(5,6))


In [4]:
Data

Galaxy,Distance,GalRa,GalDec,PosAng,Incl,MUSENatRes,MUSEPhysRes,MUSEmapNat,MUSEmap150pc,MUSE_HII_reg,IRAC_files,SFRcopt,SFR15asec,maps150,maps1kpc,CO21File,eCO21File
str7,float64,float64,float64,float64,float64,float64,int64,str71,str70,str50,str40,str58,str60,str64,str63,str72,str73
IC5332,9.01,353.61453,-36.10108,74.4,26.9,0.87,45,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/IC5332-0.87asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/IC5332-150pc_MAPS.fits,../Data/HII_reg_masks/IC5332_nebulae_mask_V2.fits,../Data/IRAC/ic5332_irac1_atGauss4.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/IC5332_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/IC5332_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/IC5332_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/IC5332_sfr_1kpc.fits,../../GalaxyFiles/ALMACO2-1/ic5332_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ic5332_12m+7m+tp_co21_150pc_broad_emom0.fits
NGC0628,9.84,24.173855,15.783643,20.7,8.9,0.92,71,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC0628-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC0628-150pc_MAPS.fits,../Data/HII_reg_masks/NGC0628_nebulae_mask_V2.fits,../Data/IRAC/ngc0628_irac1_atGauss4.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC0628_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC0628_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC0628_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC0628_sfr_1kpc.fits,../../GalaxyFiles/ALMACO2-1/ngc0628_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc0628_12m+7m+tp_co21_150pc_broad_emom0.fits
NGC1087,15.85,41.60492,-0.498717,359.1,42.9,0.92,84,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1087-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,../Data/IRAC/ngc1087_irac1_atGauss4.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC1087_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_1kpc.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_emom0.fits
NGC1300,18.99,49.920815,-19.411114,278.0,31.8,0.89,83,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1300-0.89asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1300-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1300_nebulae_mask_V2.fits,../Data/IRAC/ngc1300_irac1_atGauss4.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1300_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC1300_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1300_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1300_sfr_1kpc.fits,../../GalaxyFiles/ALMACO2-1/ngc1300_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1300_12m+7m+tp_co21_150pc_broad_emom0.fits
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,76,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,../Data/IRAC/ngc1365_irac1_atGauss4.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1365_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC1365_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1365_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1365_sfr_1kpc.fits,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_emom0.fits
NGC1385,17.22,54.369015,-24.501162,181.3,44.0,0.77,33,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1385-0.77asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1385-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1385_nebulae_mask_V2.fits,../Data/IRAC/ngc1385_irac1_atGauss4.fits,../../GalaxyFiles/MUSE

In [5]:
#Checks if Supernovae are in an image and reports back their x & y coords if in map

def check_in_image(SNras, SNdecs, SNnames, SNtypes, image, image_150, image_1kpc ,ext="HA_FLUX"):
    int_native, int_150, int_1kpc = [],[],[]
    hdulist  = pyfits.open(image)
    hdulist_150  = pyfits.open(image_150)
    hdulist_1kpc  = pyfits.open(image_1kpc)
    
    map = hdulist[1].data
    map_150 = hdulist_150[0].data
    map_1kpc = hdulist_1kpc[0].data
    
    wcs = WCS(hdulist[ext].header, naxis=2)
    naxis = wcs._naxis #size of image
    naxis1 = hdulist[ext].header['NAXIS1']
    naxis2 = hdulist[ext].header['NAXIS2']

    coords_arr = np.column_stack((SNras, SNdecs)) # ras and decs now [ra,dec]
    pix_x, pix_y = wcs.wcs_world2pix(SNras,SNdecs,0)

    #use world coordinates of all SNe to see if any fall in image (our version of footprint_contains)

    is_in_x = (pix_x >= 0) & (pix_x <= naxis[0]-1) #because of 0-indexing
    is_in_y = (pix_y >= 0) & (pix_y <= naxis[1]-1)
             
    #get the name, ra, and dec of the SNe that fall in image
    #boolean array indexing (gives back array of Trues and Falses)
    #we are pulling out the SNe that are True and assigning them to own arrays
    name_in_image = np.array(SNnames)[is_in_x & is_in_y]
    type_in_image = np.array(SNtypes)[is_in_x & is_in_y]
    ra_in_image = np.array(SNras)[is_in_x & is_in_y]
    dec_in_image = np.array(SNdecs)[is_in_x & is_in_y]

    x_coord = np.array(pix_x)[is_in_x & is_in_y]
    y_coord = np.array(pix_y)[is_in_x & is_in_y]
    
    for i in range(len(x_coord)):
        if x_coord != "[]":
            int_native.append(map[int(round(float(y_coord[i]))),int(round(float(x_coord[i])))])
            int_150.append(map_150[int(round(float(y_coord[i]))),int(round(float(x_coord[i])))])
            int_1kpc.append(map_1kpc[int(round(float(y_coord[i]))),int(round(float(x_coord[i])))])
    
    return(name_in_image, type_in_image, ra_in_image, dec_in_image, x_coord, y_coord, map, naxis1, naxis2, int_native, int_150, int_1kpc)


In [6]:
# Build MUSE Catalogue That has all supernovae within PHANGS-MUSE footprint

with open('../Data/2.Intensities.csv', 'w') as text_file:
    print('Supernova,Type,Int_native,Int_150,Int_1kpc', file = text_file)

for i in range(len(galaxies)):
    if (COPTMaps[i] != "" and os.path.isfile(COPTMaps[i])):
    
        found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, naxis1, naxis2, int_native, int_150, int_1kpc = check_in_image(ras, decs, SNe, types, COPTMaps[i], maps150[i], maps1kpc[i],ext="HA_FLUX_CORR")

        with open('../Data/2.Intensities.csv', 'a') as text_file:
            for j in range(len(found_name)):
                print(found_name[j].strip(),",", found_type[j],",", int_native[j], ",", int_150[j], ",", int_1kpc[j], file = text_file)
